In [23]:
from yelpapi import YelpAPI
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy

api_key = 'Z41R0DQquIn-_Y88pmgfAoavZA_kOFv-96EtbOcnGGdPVwqik70FkOXfIy_CRlcikfZ7nuzDtbx18DYg_vKJVJs32T3x7Zgx2y1bSxNGumP2U2S6bYtPBAOig4G3W3Yx'
yelp_api = YelpAPI(api_key)


In [2]:
# Ballparked coordinates of 4 vertices of Zip Code 60637.
upper_left_coordinate = [41.800792, -87.628796]  # [latitude, longitude]
upper_right_coordinate = [41.800792, -87.574454]
bottom_left_coordinate = [41.764310, -87.628796]
bottom_right_coordinate = [41.764310, -87.574454]
VDistance = abs(upper_left_coordinate[0] - bottom_left_coordinate[0])
HDistance = abs(upper_left_coordinate[1] - upper_right_coordinate[1])

# GPS distance to meters
# Converted using website below
# http://boulter.com/gps/distance/?from=41.800792+-87.628796&to=41.800792+-87.574454&units=k
HDistance_m = 4520  # Distance from upperleft to upperright in meters.
number_of_circles = 9
# Yelp only takes integer values for radius
circle_radius = round(HDistance_m / number_of_circles)  
circle_radius


502

In [3]:
# sample search
search_result = yelp_api.search_query(
    latitude=upper_right_coordinate[0],
    longitude=upper_right_coordinate[1],
    radius=circle_radius,
    limit=50)
df = pd.DataFrame.from_dict(search_result['businesses'], orient='columns')
df.head()


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,rating,review_count,transactions,url
0,promontory-point-chicago,"[{'alias': 'beaches', 'title': 'Beaches'}, {'a...","{'latitude': 41.7964623, 'longitude': -87.576189}",(312) 747-6620,502.463461,HQuWqZhqprjqw2oXVTcZ2A,https://s3-media1.fl.yelpcdn.com/bphoto/7c_8Pc...,False,"{'address1': '5491 S S Shore Dr', 'address2': ...",Promontory Point,+13127476620,4.5,100,[],https://www.yelp.com/biz/promontory-point-chic...


In [ ]:
# This code only goes across, not downwards.
"""
def MoveRight_map (upper_left_coordinate, upper_right_coordinate,
        number_of_circles, circle_radius):
    ''' 
        Start at the upper left corner of the grid. 
        Get the information within a certain radius. 
        Move to the left. 
        Repeat until the longitude point is reached
    '''
    distance_between_circles_h = abs (
        upper_right_coordinate[1] - upper_left_coordinate[1]) / number_of_circles
    #latitude = upper_left_coordinate[0] 
    longitude = upper_left_coordinate[1]
    
    df = pd.DataFrame()
    for i in range (number_of_circles): 
        search_result = yelp_api.search_query(latitude=latitude, 
                                        longitude=longitude,
                                        radius = circle_radius,
                                        limit=50)
        df = df.append (pd.DataFrame.from_dict(
            search_result['businesses'], orient='columns'))
        longitude += distance_between_circles_h
    return df
        
#df = MoveRight_map (upper_left_coordinate, upper_right_coordinate,
    number_of_circles=9, circle_radius=circle_radius)
df.drop_duplicates('id') # remove duplicates
"""

In [ ]:
# Code for going downwards
"""
def MoveDown (upper_left_coordinate, upper_right_coordinate, bottom_left_coordinate, 
    number_of_circles, circle_radius):
    df = pd.DataFrame()
    distance_between_circles_v = abs (
        upper_left_coordinate[0] - bottom_left_coordinate[0]) / number_of_circles
    latitude = upper_left_coordinate[0] 
    for i in range (number_of_circles):
        df = df.append (MoveRight_map (upper_left_coordinate, upper_right_coordinate, 
                            number_of_circles, circle_radius))
        latitude -= distance_between_circles_v
    return df
"""

In [15]:
def MoveMap(upper_left_coordinate, upper_right_coordinate, bottom_left_coordinate,
            number_of_circles, circle_radius,
            category):
    '''
        Start at the upper left corner of the grid.
        Get the information within a certain radius.
        Move to the left.
        Repeat until the upper right point is reached.
        Move back to to the longitude of upper left point.
        Move downwards.
        Repeat.
    '''
    distance_between_circles_h = abs(
        upper_right_coordinate[1] - upper_left_coordinate[1]) / number_of_circles
    distance_between_circles_v = abs(
        upper_left_coordinate[0] - bottom_left_coordinate[0]) / number_of_circles
    latitude = upper_left_coordinate[0]
    longitude = upper_left_coordinate[1]

    df = pd.DataFrame()
    for v_step in range(number_of_circles):
        for h_step in range(number_of_circles):
            search_result = yelp_api.search_query(
                term=category,
                latitude=latitude,
                longitude=longitude,
                radius=circle_radius,
                limit=50)
            normalize = pd.DataFrame.from_dict(
                json_normalize(search_result['businesses']), orient='columns')
            # df = df.append (pd.DataFrame.from_dict(dfadd, orient='columns'))
            df = df.append(normalize)
            longitude += distance_between_circles_h
        longitude = upper_left_coordinate[1]
        latitude -= distance_between_circles_v
    return df.drop_duplicates(['id']).reset_index().drop('index',axis=1)


In [10]:
data_in_60637 = MoveMap(upper_left_coordinate, upper_right_coordinate, bottom_left_coordinate,
              number_of_circles=9, circle_radius=circle_radius,
              category='')


In [11]:
data_in_60637

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,jj-fish-and-chicken-chicago-3,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",41.796920,-87.630130,(773) 268-1600,535.446583,53z65ak2aZ2zrQEGAvq7NQ,https://s3-media3.fl.yelpcdn.com/bphoto/Oi2mxI...,False,5401 S Wentworth Ave,...,"[5401 S Wentworth Ave, Chicago, IL 60609]",IL,60609,JJ Fish and Chicken,+17732681600,$$,4.5,9,[],https://www.yelp.com/biz/jj-fish-and-chicken-c...
1,checkers-chicago,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",41.794779,-87.630321,(773) 268-5145,663.447196,hNxONY8We4_YEQc_p2WcBg,https://s3-media2.fl.yelpcdn.com/bphoto/g-U5QE...,False,5451 S Wentworth Ave,...,"[5451 S Wentworth Ave, Chicago, IL 60609]",IL,60609,Checkers,+17732685145,$,2.5,28,[],https://www.yelp.com/biz/checkers-chicago?adju...
2,waynes-bar-b-que-and-cajun-chicago,"[{'alias': 'chinese', 'title': 'Chinese'}]",41.796920,-87.630130,(773) 536-2282,572.995547,bmFyjzhFHdm1JL2Lx1iwqA,https://s3-media1.fl.yelpcdn.com/bphoto/Ft4B3V...,False,5401 S Wentworth Ave,...,"[5401 S Wentworth Ave, Chicago, IL 60609]",IL,60609,Wayne's Bar-B-Que and Cajun,+17735362282,$,4.0,6,[],https://www.yelp.com/biz/waynes-bar-b-que-and-...
3,new-grand-chinese-kitchen-chicago,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",41.796920,-87.630130,(773) 536-8282,534.058034,LPFuagNX9Petb8vxMN42Ww,https://s3-media4.fl.yelpcdn.com/bphoto/5aAgCq...,False,5401 S Wentworth Ave,...,"[5401 S Wentworth Ave, Chicago, IL 60609]",IL,60609,New Grand Chinese Kitchen,+17735368282,$,3.0,10,"[pickup, delivery]",https://www.yelp.com/biz/new-grand-chinese-kit...
4,pizza-hut-chicago-33,"[{'alias': 'pizza', 'title': 'Pizza'}]",41.796920,-87.630130,(773) 285-5056,534.058034,5SFHn1n2oQfRiyo9n456sg,,False,5401 S Wentworth Ave,...,"[5401 S Wentworth Ave, Chicago, IL 60609]",IL,60609,Pizza Hut,+17732855056,NaN,3.0,3,[delivery],https://www.yelp.com/biz/pizza-hut-chicago-33?...
5,dunkin-donuts-chicago-317,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",41.794880,-87.626050,(773) 558-6695,694.567600,bvH914f-picmzvjuGg_O7w,,False,5454 S State St,...,"[5454 S State St, Chicago, IL 60609]",IL,60609,Dunkin' Donuts,+17735586695,NaN,3.0,1,[],https://www.yelp.com/biz/dunkin-donuts-chicago...
6,friistyle-chicago,"[{'alias': 'tradamerican', 'title': 'American ...",41.802814,-87.619239,(773) 548-5375,347.066085,WwmxVYk-DYS76iZ3ey1ybQ,https://s3-media2.fl.yelpcdn.com/bphoto/1rGYvJ...,False,5059 S Prairie,...,"[5059 S Prairie, Chicago, IL 60615]",IL,60615,Friistyle,+17735485375,NaN,4.5,37,[],https://www.yelp.com/biz/friistyle-chicago?adj...
7,ms-biscuit-chicago,"[{'alias': 'breakfast_brunch', 'title': 'Break...",41.795540,-87.623620,(773) 268-8088,592.339099,xHxxDdnfdiSvxYcwdeokaQ,https://s3-media4.fl.yelpcdn.com/bphoto/vO1AMe...,False,5431 S Wabash Ave,...,"[5431 S Wabash Ave, Chicago, IL 60615]",IL,60615,Ms Biscuit,+17732688088,$$,3.0,190,[],https://www.yelp.com/biz/ms-biscuit-chicago?ad...
8,love-taco-chicago,"[{'alias': 'tacos', 'title': 'Tacos'}]",41.801983,-87.622360,(312) 650-9635,136.455893,vTF9BQt2AwQGW6CnZSwzQA,https://s3-media2.fl.yelpcdn.com/bphoto/Y_rHh7...,False,111 East 51st St,...,"[111 East 51st St, Chicago, IL 60615]",IL,60615,Love Taco,+13126509635,$,3.0,51,"[delivery, pickup]",https://www.yelp.com/biz/love-taco-chicago?adj...
9,skidoes-cafe-and-grill-chicago,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ...",41.801917,-87.620230,(773) 690-5287,272.649682,BTWr_cMM_uac4v0y9n7kVA,https://s3-media4.fl.yelpcdn.com/bphoto/PKZbTB...,False,235 E 51st St,...,"[235 E 51st St, Chicago, IL 60615]",IL,60615,Skidoe's Cafe and Grill,+17736905287,$$,3.5,18,"[delivery, pickup]",https://www.yelp.com/biz/skidoes-cafe-and-gril...


In [7]:
def get_yelp_data_by_location(location, number_of_calls):
    '''
        Given a location (ex. ZipCode),
        use Yelp API to retrieve data.
        Repeat by number of calls.
        Returns a dataframe.
    '''
    df = pd.DataFrame()
    for call in range (number_of_calls):
        search_result = yelp_api.search_query(
                    location=location,
                    limit=50)
        normalize = pd.DataFrame.from_dict(
                    json_normalize(search_result['businesses']), orient='columns')
        df = df.append(normalize)
    return df

location_data = get_yelp_data_by_location (
                    location='60637',
                    number_of_calls=2)
location_data.drop_duplicates(['id']).head()
# Calling any amount of times in the same location produces same results

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,museum-of-science-and-industry-chicago-3,"[{'alias': 'museums', 'title': 'Museums'}]",41.790577,-87.583069,(773) 684-1414,1620.209142,X7inrhYbhtjgJ4MrwtPlJQ,https://s3-media3.fl.yelpcdn.com/bphoto/EuFRTd...,False,5700 S Lake Shore Dr,...,"[5700 S Lake Shore Dr, Chicago, IL 60637]",IL,60637,Museum of Science and Industry,+17736841414,NaN,4.5,1022,[],https://www.yelp.com/biz/museum-of-science-and...
1,medici-on-57th-chicago,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",41.791267,-87.593766,(773) 667-7394,1039.364699,Zx87qmviYtwDOyBJah213g,https://s3-media2.fl.yelpcdn.com/bphoto/lzEU2Z...,False,1327 E 57th St,...,"[1327 E 57th St, Chicago, IL 60637]",IL,60637,Medici On 57th,+17736677394,$$,3.5,748,[],https://www.yelp.com/biz/medici-on-57th-chicag...
2,oriental-institute-museum-chicago-2,"[{'alias': 'museums', 'title': 'Museums'}]",41.789218,-87.597814,(773) 702-9520,702.046671,NfnmLKxavg9pFBgUxgAkJA,https://s3-media2.fl.yelpcdn.com/bphoto/1gZNys...,False,1155 E 58th St,...,"[1155 E 58th St, Chicago, IL 60637]",IL,60637,Oriental Institute Museum,+17737029520,NaN,4.5,77,[],https://www.yelp.com/biz/oriental-institute-mu...
3,valois-chicago,"[{'alias': 'breakfast_brunch', 'title': 'Break...",41.799602,-87.588372,(773) 667-0647,2075.136221,BwkeC6JB_4fIVpth2VUyQA,https://s3-media4.fl.yelpcdn.com/bphoto/ixqwiM...,False,1518 E 53rd St,...,"[1518 E 53rd St, Chicago, IL 60615]",IL,60615,Valois,+17736670647,$,4.0,513,[],https://www.yelp.com/biz/valois-chicago?adjust...
4,robust-coffee-lounge-chicago,"[{'alias': 'breakfast_brunch', 'title': 'Break...",41.780501,-87.596232,(773) 891-4240,414.327526,-Tp77Ce5OCpP1-KXUYtX9A,https://s3-media1.fl.yelpcdn.com/bphoto/ayXC_8...,False,6300 S Woodlawn,...,"[6300 S Woodlawn, Chicago, IL 60637]",IL,60637,Robust Coffee Lounge,+17738914240,$,4.0,211,"[pickup, delivery]",https://www.yelp.com/biz/robust-coffee-lounge-...


In [ ]:
data_in_60637.to_csv ('Businesses in 60637.csv')

In [16]:
cafe_in_60637 = MoveMap(upper_left_coordinate, upper_right_coordinate, bottom_left_coordinate,
              number_of_circles=9, circle_radius=circle_radius,
              category='cafe')

In [17]:
cafe_in_60637

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,dunkin-donuts-chicago-317,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",41.794880,-87.626050,(773) 558-6695,694.567600,bvH914f-picmzvjuGg_O7w,,False,5454 S State St,...,"[5454 S State St, Chicago, IL 60609]",IL,60609,Dunkin' Donuts,+17735586695,NaN,3.0,1,[],https://www.yelp.com/biz/dunkin-donuts-chicago...
1,mcdonalds-chicago-70,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",41.795018,-87.622940,(773) 924-0821,642.216032,xJPhx0iI_ix760SUHf4DCw,https://s3-media2.fl.yelpcdn.com/bphoto/Wq18Xx...,False,70 E Garfield,...,"[70 E Garfield, Chicago, IL 60615]",IL,60615,McDonald's,+17739240821,$,1.0,16,[],https://www.yelp.com/biz/mcdonalds-chicago-70?...
2,teamo-boba-bar-chicago-3,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...",41.794885,-87.599060,(773) 947-6547,671.662851,TZyMOAThow2bJqXcvaJAAg,https://s3-media1.fl.yelpcdn.com/bphoto/3L8mB9...,False,1115 E 55th St,...,"[1115 E 55th St, Chicago, IL 60615]",IL,60615,Te'amo Boba Bar,+17739476547,$$,4.0,32,[],https://www.yelp.com/biz/teamo-boba-bar-chicag...
3,starbucks-chicago-122,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",41.795161,-87.597232,(773) 256-1930,633.596263,1lHVis5d8vpd72vx0cjiOg,https://s3-media3.fl.yelpcdn.com/bphoto/JCbxQD...,False,1174 E 55th St,...,"[1174 E 55th St, Chicago, IL 60615]",IL,60615,Starbucks,+17732561930,$,3.5,47,[],https://www.yelp.com/biz/starbucks-chicago-122...
4,cafe-53-chicago,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",41.799359,-87.592395,(773) 493-1000,160.036360,_SSK3MyOgE-dYsE4XUUbIQ,https://s3-media1.fl.yelpcdn.com/bphoto/-phGN8...,False,1369 E 53rd St,...,"[1369 E 53rd St, Chicago, IL 60615]",IL,60615,Cafe 53,+17734931000,$,4.0,160,"[delivery, pickup]",https://www.yelp.com/biz/cafe-53-chicago?adjus...
5,einstein-bros-bagels-chicago-23,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",41.800150,-87.588360,(773) 947-0485,356.041458,SS5HzgdYVu3wcdiep5nujA,https://s3-media1.fl.yelpcdn.com/bphoto/Cj_BJM...,False,5225 S Harper Ave,...,"[5225 S Harper Ave, Harper Court, Chicago, IL ...",IL,60615,Einstein Bros Bagels,+17739470485,NaN,2.5,12,[],https://www.yelp.com/biz/einstein-bros-bagels-...
6,starbucks-chicago-195,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",41.799742,-87.588947,(773) 324-1241,322.040055,urZt9uX3Y98oZgXfZRzaWw,https://s3-media3.fl.yelpcdn.com/bphoto/V1o6AK...,False,1530 E 53rd St,...,"[1530 E 53rd St, Chicago, IL 60615]",IL,60615,Starbucks,+17733241241,$$,3.0,41,[],https://www.yelp.com/biz/starbucks-chicago-195...
7,dunkin-donuts-baskin-robbins-chicago-31,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",41.799642,-87.590910,(773) 288-5719,152.566840,d-KJ3lLU1G16A9vnE--7yQ,https://s3-media2.fl.yelpcdn.com/bphoto/hyK0Xk...,False,1418 E 53rd St,...,"[1418 E 53rd St, Ste 5, Chicago, IL 60615]",IL,60615,Dunkin' Donuts / Baskin-Robbins,+17732885719,$,2.5,45,[],https://www.yelp.com/biz/dunkin-donuts-baskin-...
8,sip-and-savor-chicago,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",41.799571,-87.583865,(773) 952-4532,259.277354,WKSBC4wJcQJJoEKri-L3Dw,https://s3-media1.fl.yelpcdn.com/bphoto/ftlk76...,False,5301 S Hyde Park Blvd,...,"[5301 S Hyde Park Blvd, Chicago, IL 60615]",IL,60615,Sip & Savor,+17739524532,$,4.0,109,[],https://www.yelp.com/biz/sip-and-savor-chicago...
9,bridgeport-coffee-chicago-2,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",41.803165,-87.586702,(773) 947-0348,264.304131,9TgZHXAB1K0iiI4X2vY6MQ,https://s3-media2.fl.yelpcdn.com/bphoto/GGYlG3...,False,5030 S Cornell Ave,...,"[5030 S Cornell Ave, Chicago, IL 60615]",IL,60615,Bridgeport Coffee,+17739470348,$,4.0,61,[],https://www.yelp.com/biz/bridgeport-coffee-chi...


In [18]:
# All these numbers are overestimates.

# Ballparked coordinates of 4 vertices of Chicago
chi_u_l = [42.031355, -87.946627] # [latitude, longitude]
chi_u_r = [42.031355, -87.512777]
chi_b_l = [41.633678, -87.946627]
chi_b_r = [41.633678, -87.512777]
"""
    About 36 km from u_l to u_r
    About 45 km from u_l to b_l
    Area is about: 1620 km^2 (1.62e+9 m^2)
    Based on Yelp(https://www.yelp.com/search?find_desc=cafe&find_loc=Chicago%2C+IL&ns=1):
    There are about 2500 cafes in the Chicago area
    (possibly up to 7500 in the grid I chose because the area is less than 3 times bigger).
    That means about 1 cafe every 216000 m^2 (result seems off),
    or 50 cafes every 10800000 m^2.
    Then the radius I should pick is 1900 meters.
    45/1.9 = 24, so the number of circles should be 24
"""
cafe_in_chicago = MoveMap(chi_u_l, chi_u_r, chi_b_l,
              number_of_circles=24, circle_radius=1900,
              category='cafe')

In [21]:
cafe_in_chicago.to_csv ('cafe_in_chicago.csv')
cafe_in_chicago.head()

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,jelly-cafe-mount-prospect,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",42.048766,-87.962212,(847) 979-8989,2324.740179,n6zMFWY4VmH0OWkmNpTi2w,https://s3-media4.fl.yelpcdn.com/bphoto/DvVpm0...,False,1784 W Golf Rd,...,"[1784 W Golf Rd, Mount Prospect, IL 60056]",IL,60056,Jelly Cafe,+18479798989,$$,4.0,312,[],https://www.yelp.com/biz/jelly-cafe-mount-pros...
1,ball-factory-mount-prospect-mount-prospect-2,"[{'alias': 'indoor_playcenter', 'title': 'Indo...",42.046538,-87.940895,(847) 378-8706,1626.749441,23wCAartGFdQG78F0C85Nw,https://s3-media2.fl.yelpcdn.com/bphoto/mKgLX_...,False,1042 S Elmhurst Rd,...,"[1042 S Elmhurst Rd, Mount Prospect, IL 60056]",IL,60056,Ball Factory Mount Prospect,+18473788706,NaN,4.5,25,[],https://www.yelp.com/biz/ball-factory-mount-pr...
2,starbucks-elk-grove-village-4,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",42.012537,-87.959825,(847) 228-5212,2419.466134,GPDeBx49eV-0izvWIuhMWg,https://s3-media3.fl.yelpcdn.com/bphoto/7vE4AY...,False,934 Busse Rd,...,"[934 Busse Rd, Elk Grove Village, IL 60007]",IL,60007,Starbucks,+18472285212,$$,4.0,18,[],https://www.yelp.com/biz/starbucks-elk-grove-v...
3,dunkin-donuts-des-plaines,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",42.035950,-87.941040,(847) 981-9022,696.474416,Y8HoEtv-X9jY5Ydx-kWPPQ,https://s3-media2.fl.yelpcdn.com/bphoto/6EitZB...,False,850 Elmhurst Rd,...,"[850 Elmhurst Rd, Des Plaines, IL 60016]",IL,60016,Dunkin' Donuts,+18479819022,$,2.5,12,[],https://www.yelp.com/biz/dunkin-donuts-des-pla...
4,7-eleven-des-plaines-2,"[{'alias': 'convenience', 'title': 'Convenienc...",42.022252,-87.941299,(847) 364-6994,1103.711754,C-wyvJB9WPdRxcZg1kJwrw,https://s3-media4.fl.yelpcdn.com/bphoto/nEOsB_...,False,825 W Oakton,...,"[825 W Oakton, Des Plaines, IL 60018]",IL,60018,7-Eleven,+18473646994,$,2.0,5,[],https://www.yelp.com/biz/7-eleven-des-plaines-...


In [24]:
"""
    Cutting down coordinates, 
    increasing number of circles,
    and decreasing circle size
"""
chi_u_l = [42.030116, -87.946627] # [latitude, longitude]
chi_u_r = [42.030116, -87.512777]
chi_b_l = [41.633678, -87.946627]
chi_b_r = [41.633678, -87.512777]

cafe_in_chicago = MoveMap(chi_u_l, chi_u_r, chi_b_l,
              number_of_circles=45, circle_radius=1000,
              category='cafe')

In [26]:
cafe_in_chicago.to_csv ('cafe_in_chicago.csv')
cafe_in_chicago.head()

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,dunkin-donuts-des-plaines,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",42.035950,-87.941040,(847) 981-9022,806.918670,Y8HoEtv-X9jY5Ydx-kWPPQ,https://s3-media2.fl.yelpcdn.com/bphoto/6EitZB...,False,850 Elmhurst Rd,...,"[850 Elmhurst Rd, Des Plaines, IL 60016]",IL,60016,Dunkin' Donuts,+18479819022,$,2.5,12,[],https://www.yelp.com/biz/dunkin-donuts-des-pla...
1,7-eleven-des-plaines-2,"[{'alias': 'convenience', 'title': 'Convenienc...",42.022252,-87.941299,(847) 364-6994,978.911966,C-wyvJB9WPdRxcZg1kJwrw,https://s3-media4.fl.yelpcdn.com/bphoto/nEOsB_...,False,825 W Oakton,...,"[825 W Oakton, Des Plaines, IL 60018]",IL,60018,7-Eleven,+18473646994,$,2.0,5,[],https://www.yelp.com/biz/7-eleven-des-plaines-...
2,dunkin-donuts-baskin-robbins-des-plaines-3,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",42.022729,-87.939152,,1027.589520,3CZL6NXqNPjoJukz1Xn01g,https://s3-media3.fl.yelpcdn.com/bphoto/Ip7mKG...,False,782 W Oakton St,...,"[782 W Oakton St, Ste I, Des Plaines, IL 60018]",IL,60018,Dunkin' Donuts / Baskin-Robbins,,$$,2.5,7,[],https://www.yelp.com/biz/dunkin-donuts-baskin-...
3,7-eleven-des-plaines-4,"[{'alias': 'convenience', 'title': 'Convenienc...",42.023818,-87.909327,(847) 297-0285,984.412120,f5T9pP37sdCq2QDcSEZ8pg,https://s3-media3.fl.yelpcdn.com/bphoto/PnRaYz...,False,586 E Oakton St,...,"[586 E Oakton St, Des Plaines, IL 60018]",IL,60018,7-Eleven,+18472970285,$,2.0,3,[],https://www.yelp.com/biz/7-eleven-des-plaines-...
4,bs-sweet-bites-des-plaines,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",42.022464,-87.897392,(847) 795-0743,854.522067,Ss91O4BF-0dSqXqZIuDBYA,https://s3-media2.fl.yelpcdn.com/bphoto/x3dWUo...,False,1641 Oakton Pl,...,"[1641 Oakton Pl, Des Plaines, IL 60018]",IL,60018,B's Sweet Bites,+18477950743,$$,4.0,67,[],https://www.yelp.com/biz/bs-sweet-bites-des-pl...


In [29]:
cafe_in_chicago.shape[0] # 1595. The one before that had 1300 cafes.

1595